In [1]:
import numpy as np
import pandas as pd
# from nltk import bigrams,trigrams
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report
from textblob import TextBlob
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.svm import SVC

In [2]:
from tensorflow.keras import datasets # https://www.tensorflow.org/api_docs/python/tf/keras/datasets
from tensorflow.keras import layers # https://www.tensorflow.org/api_docs/python/tf/keras/layers
from tensorflow.keras import activations # https://www.tensorflow.org/api_docs/python/tf/keras/activations
from tensorflow.keras import initializers # https://www.tensorflow.org/api_docs/python/tf/keras/initializers
from tensorflow.keras import losses # https://www.tensorflow.org/api_docs/python/tf/keras/losses
from tensorflow.keras import metrics # https://www.tensorflow.org/api_docs/python/tf/keras/metrics
from tensorflow.keras import optimizers # https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
from tensorflow.keras import regularizers # https://www.tensorflow.org/api_docs/python/tf/keras/regularizers
from tensorflow.keras.optimizers import schedules # https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules
from tensorflow.keras import callbacks # https://www.tensorflow.org/api_docs/python/tf/keras/callbacks
from tensorflow.keras import utils # https://www.tensorflow.org/api_docs/python/tf/keras/utils
from tensorflow.keras import models # https://www.tensorflow.org/api_docs/python/tf/keras/models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
#reading all the 17 features
all_features = pd.read_csv("featuresAll.csv")
labels = pd.read_csv("labels.csv")
all_features.head()

,polarity,subjectivity,wordCount,stdrating,clusterid,meanSimilarity,varOfSimilarity,meanRating,numberOfReviews,avgNoOfWords,totalReviews,isSingleton,posReviewRatio,negReviewRatio,avgNoOfReviews,maxNoOfReviews,dateVariance
0,0.195833,0.395833,143,0.896514,141,0.043510,0.005451,4.009524,210,37.410256,39,0,0.974359,0.025641,1.625,3,1.360395e+02
1,0.025000,0.650000,163,0.896514,74,0.067578,0.005659,4.009524,210,27.000000,1,1,1.000000,0.000000,1.000,1,2.003740e+06
2,0.220000,0.328718,115,0.896514,167,0.047143,0.005273,4.009524,210,22.000000,2,0,1.000000,0.000000,2.000,2,0.000000e+00
3,0.555134,0.776786,315,0.896514,378,0.076343,0.005720,4.009524,210,52.000000,1,1,1.000000,0.000000,1.000,1,2.003740e+06
4,0.138715,0.538294,420,0.896514,992,0.062976,0.005454,4.009524,210,86.200000,5,0,0.800000,0.200000,1.000,1,2.618750e+01


In [4]:
all_features['labels'] = labels

In [5]:
only_one = all_features.loc[all_features['labels'] == 1]
only_zero = all_features.loc[all_features['labels'] == 0]
train_one = only_one.iloc[:29508]
test_one = only_one.iloc[29508:]
train_zero = only_zero.iloc[:257733]
test_zero = only_zero.iloc[257733:]
train = train_one.append(train_zero, ignore_index = True)
test = test_one.append(test_zero, ignore_index = True)
train = train.sample(frac=1).reset_index(drop=True)
X_train = train.iloc[:,:17]
Y_train = train.iloc[:,17]
X_test = test.iloc[:,:17]
Y_test = test.iloc[:,17]
X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()
X_test = X_test.to_numpy()
Y_test = Y_test.to_numpy()
# np.random.shuffle(train)
# np.random.shuffle(test)

In [4]:
# all_features = all_features.to_numpy()
# print(all_features.shape)

(359052, 17)


In [5]:
# #reading the reviews
# file1 = open("YelpNYC/reviewContent")
# all_stop_words = stopwords.words('english')
# data1 = []
# for s in file1:
#     row = s[17:]
#     row = re.sub('[^a-zA-Z0-9 \n\..]', '', row)
#     row = ' '.join([word for word in row.split() if word not in all_stop_words])
#     data1.append(row)
# file1.close()
# print(len(data1))

359052


In [5]:
# #reading all the labels
# labels = pd.read_csv("labels.csv")
# Y = labels.to_numpy()
# Y.shape

(359052, 1)

## TF-IDF Vectorization

In [7]:
# #using tfidf vectorization to convert the reviews into features
# tfidf_vector = TfidfVectorizer(stop_words='english')
# X_tfidf = tfidf_vector.fit_transform(data1)

In [8]:
# #reducing the number of features
# svd = TruncatedSVD(n_components = 100)
# X_tfidf = svd.fit_transform(X_tfidf)

In [6]:
# #putting all the features together
# # X1 = np.hstack((X_tfidf,all_features))
# X1 = all_features
# X1.shape

(359052, 17)

In [6]:
model = tf.keras.Sequential([

    
    layers.Flatten(input_dim = len(X_train[0,:])), 
    
    layers.Dense(units=64, activation=None), 
    layers.BatchNormalization(),
    layers.Activation(activations.relu),
    
    layers.Dense(units=128, activation=None), 
    layers.BatchNormalization(),
    layers.Activation(activations.relu),
    
    layers.Dense(units=128, activation=None), 
    layers.BatchNormalization(),
    layers.Activation(activations.relu),
    
    layers.Dense(units=32, activation=None), 
    layers.BatchNormalization(),
    layers.Activation(activations.relu),
    
    layers.Dense(units=1, activation=activations.sigmoid) 
])

model.summary()

model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001), 
    loss=losses.BinaryCrossentropy(), 
    metrics=['accuracy']
) 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 17)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                1152      
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0

In [7]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

In [8]:
model.fit(x=X_train, y=Y_train, epochs=10,batch_size=128, verbose=1,class_weight={0:1,1:7})

Epoch 1/10
2245/2245 [==============================] - 10s 3ms/step - loss: 0.9595 - accuracy: 0.7292
Epoch 2/10
2245/2245 [==============================] - 7s 3ms/step - loss: 0.9838 - accuracy: 0.7339
Epoch 3/10
2245/2245 [==============================] - 7s 3ms/step - loss: 1.0018 - accuracy: 0.7348
Epoch 4/10
2245/2245 [==============================] - 7s 3ms/step - loss: 0.9897 - accuracy: 0.7330
Epoch 5/10
2245/2245 [==============================] - 7s 3ms/step - loss: 0.9858 - accuracy: 0.7365
Epoch 6/10
2245/2245 [==============================] - 7s 3ms/step - loss: 0.9973 - accuracy: 0.7331
Epoch 7/10
2245/2245 [==============================] - 7s 3ms/step - loss: 0.9883 - accuracy: 0.7344
Epoch 8/10
2245/2245 [==============================] - 7s 3ms/step - loss: 0.9915 - accuracy: 0.7350
Epoch 9/10
2245/2245 [==============================] - 7s 3ms/step - loss: 0.9806 - accuracy: 0.7359
Epoch 10/10
2245/2245 [==============================] - 8s 3ms/step - loss: 0.98

In [9]:
predictions = np.round(model.predict(X_test))


print(Y_test.shape)
print(classification_report(Y_test,predictions))

(71811,)
              precision    recall  f1-score   support

           0       0.95      0.74      0.83     64434
           1       0.23      0.66      0.34      7377

    accuracy                           0.73     71811
   macro avg       0.59      0.70      0.58     71811
weighted avg       0.88      0.73      0.78     71811



In [14]:
# type(predictions)

numpy.ndarray

In [10]:
real_one = 0
real_zero = 0
pred_one = 0
pred_zero = 0
for i in range(len(predictions)):
    if Y_test[i] == 1:
        real_one += 1
        if predictions[i] == 1:
            pred_one += 1
    else:
        real_zero += 1
        if predictions[i] == 0:
            pred_zero += 1
print("Accuracy for fake reviews in ANN: ",pred_one/real_one)
print("Accuracy for true reviews in ANN: ",pred_zero/real_zero)

Accuracy for fake reviews in ANN:  0.6550088111698522
Accuracy for true reviews in ANN:  0.7432256262221808


In [11]:
# perm = PermutationImportance(model).fit(X_train, Y_train)
# eli5.show_weights(perm, feature_names = X_train.columns.tolist())

In [ ]:
# svc = SVC(kernel='poly')
# svc.fit(X_train,Y_train)

In [8]:
# models1 = tf.keras.Sequential([

    
#     layers.Flatten(input_dim = len(X1[0,:])), 
    
#     layers.Dense(units=64, activation=None), 
#     layers.BatchNormalization(),
#     layers.Activation(activations.relu),
    
#     layers.Dense(units=128, activation=None), 
#     layers.BatchNormalization(),
#     layers.Activation(activations.relu),
    
#     layers.Dense(units=128, activation=None), 
#     layers.BatchNormalization(),
#     layers.Activation(activations.relu),
    
#     layers.Dense(units=32, activation=None), 
#     layers.BatchNormalization(),
#     layers.Activation(activations.relu),
    
#     layers.Dense(units=1, activation=activations.sigmoid) 
# ])

# models1.summary()

# models1.compile(
#     optimizer=optimizers.Adam(learning_rate=0.001), 
#     loss=losses.BinaryCrossentropy(), 
#     metrics=['accuracy']
# ) 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 17)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                1152      
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0

In [13]:
# skf = StratifiedKFold(n_splits=5)
# fold_no = 1
# for train_index, test_index in skf.split(X1, Y):
#     X_train = X1[train_index,:]
#     X_test = X1[test_index,:]
#     Y_train = Y[train_index,:]
#     Y_test = Y[test_index,:]
#     model.fit(X_train, Y_train, epochs=5,batch_size=128,)
#     predictions = model.predict(X_test)
#     print('Fold',str(fold_no),'Accuracy:',accuracy_score(Y_test,np.round(predictions)), ' Precision: ',precision_score(Y_test,np.round(predictions)), ' Recall: ',recall_score(Y_test,np.round(predictions)), ' F1-Score: ',f1_score(Y_test,np.round(predictions)))
#     fold_no += 1

Epoch 1/5
2245/2245 [==============================] - 5s 2ms/step - loss: 680.1175 - accuracy: 0.8246
Epoch 2/5
2245/2245 [==============================] - 4s 2ms/step - loss: 3.6577 - accuracy: 0.8796
Epoch 3/5
2245/2245 [==============================] - 5s 2ms/step - loss: 0.7007 - accuracy: 0.8971
Epoch 4/5
2245/2245 [==============================] - 4s 2ms/step - loss: 1.0891 - accuracy: 0.8973
Epoch 5/5
2245/2245 [==============================] - 4s 2ms/step - loss: 1.2254 - accuracy: 0.8973
Fold 1 Accuracy: 0.8962693737728203  Precision:  0.3125  Recall:  0.008133387555917039  F1-Score:  0.015854141894569955
Epoch 1/5
2245/2245 [==============================] - 4s 2ms/step - loss: 1.3161 - accuracy: 0.8964
Epoch 2/5
2245/2245 [==============================] - 4s 2ms/step - loss: 1.4950 - accuracy: 0.8965
Epoch 3/5
2245/2245 [==============================] - 4s 2ms/step - loss: 1.0375 - accuracy: 0.8966
Epoch 4/5
2245/2245 [==============================] - 5s 2ms/step - l

/home/sabu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2245/2245 [==============================] - 4s 2ms/step - loss: 0.6181 - accuracy: 0.8954
Epoch 2/5
2245/2245 [==============================] - 4s 2ms/step - loss: 0.5867 - accuracy: 0.8966
Epoch 3/5
2245/2245 [==============================] - 4s 2ms/step - loss: 0.4439 - accuracy: 0.8971
Epoch 4/5
2245/2245 [==============================] - 4s 2ms/step - loss: 0.4300 - accuracy: 0.8971
Epoch 5/5
2245/2245 [==============================] - 4s 2ms/step - loss: 0.4268 - accuracy: 0.8972
Fold 4 Accuracy: 0.8972705751288121  Precision:  0.0  Recall:  0.0  F1-Score:  0.0
Epoch 1/5
2245/2245 [==============================] - 3s 2ms/step - loss: 0.4926 - accuracy: 0.8967
Epoch 2/5
2245/2245 [==============================] - 3s 2ms/step - loss: 0.5177 - accuracy: 0.8970
Epoch 3/5
2245/2245 [==============================] - 3s 2ms/step - loss: 0.3998 - accuracy: 0.8970
Epoch 4/5
2245/2245 [==============================] - 3s 2ms/step - loss: 0.4625 - accuracy: 0.8968
Epoch 5/5
2245/224

# 2-gram vectorization

In [7]:
# bigram_vector = CountVectorizer(ngram_range=(2, 2),token_pattern=r'\b\w+\b', min_df=1)
# X_ngram = bigram_vector.fit_transform(data1)

In [ ]:
# svd2 = TruncatedSVD(n_components = 100)
# X_ngram = svd2.fit_transform(X_ngram)

In [ ]:
# X2 = np.hstack((X_ngram,all_features))
# X2.shape

In [ ]:
# models2 = tf.keras.Sequential([

    
#     layers.Flatten(input_dim = len(X1[0,:])), 
    
#     layers.Dense(units=256, activation=None), 
#     layers.BatchNormalization(),
#     layers.Activation(activations.relu),
    
#     layers.Dense(units=128, activation=None), 
#     layers.BatchNormalization(),
#     layers.Activation(activations.relu),
    
#     layers.Dense(units=1, activation=activations.sigmoid) 
# ])

# models2.summary()

# models2.compile(
#     optimizer=optimizers.Adam(learning_rate=0.001), 
#     loss=losses.BinaryCrossentropy(), 
#     metrics=['accuracy']
# ) 

In [ ]:
# skf = StratifiedKFold(n_splits=5)
# fold_no = 1
# for train_index, test_index in skf.split(X2, Y):
#     X_train = X2[train_index,:]
#     X_test = X2[test_index,:]
#     Y_train = Y[train_index,:]
#     Y_test = Y[test_index,:]
#     history1 = models1.fit(X_train, Y_train, epochs=5,batch_size=128,)
#     predictions = models1.predict(X_test)
#     print('Fold',str(fold_no),'Accuracy:',accuracy_score(Y_test,np.round(predictions)), ' Precision: ',precision_score(Y_test,np.round(predictions)), ' Recall: ',recall_score(Y_test,np.round(predictions)), ' F1-Score: ',f1_score(Y_test,np.round(predictions)))
#     fold_no += 1=0.1,
# ) 